In [31]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process


[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [32]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

point_projections = generate_projections()
stat_projections = generate_stat_projections()

projections_data = append_stat_projections(point_projections, stat_projections, 2)

In [33]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


# 2024/25 GW2 Challenge: All Out Attack - Goals are worth double points. Your squad may only contain 1 Goalkeeper and 4 Forwards.

### Points Refinement

In [34]:
# Refine points values based on scoring probabilities
# Function to calculate the adjusted points for Forwards
def adjust_forward_points(row):
    if row['Position'] == 'Forward':
        # Assume that a goal is worth 4 points in the original prediction
        estimated_goal_points = row['Score'] * 4
        
        # Remove the original goal points from the prediction
        non_goal_points = row['Predicted_Points'] - estimated_goal_points
        
        # Double the goal points
        doubled_goal_points = estimated_goal_points * 2
        
        # Return the adjusted total
        return non_goal_points + doubled_goal_points
    else:
        # For non-Forwards, return the original prediction
        return row['Predicted_Points']

# Apply the adjustment to the dataframe
projections_data['Adjusted_Points'] = projections_data.apply(adjust_forward_points, axis=1)

# Round the adjusted points to 2 decimal places
projections_data['Adjusted_Points'] = projections_data['Adjusted_Points'].round(2)
projections_data['Predicted_Points'] = projections_data['Adjusted_Points']

### Optimisation

In [35]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-gw1-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5 (1 goalkeeper + 4 forwards)
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Exactly 4 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) == 4

# Budget constraint: Total cost must be less than or equal to 35m
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 35

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/5bfbf99aa5c64adebd87aa88654ed279-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/91/3jnk9jzn3kx1mwsfzhddwbww0000gn/T/5bfbf99aa5c64adebd87aa88654ed279-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 318 COLUMNS
At line 3663 RHS
At line 3977 BOUNDS
At line 4594 ENDATA
Problem MODEL has 313 rows, 616 columns and 1626 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 41.2757 - 0.00 seconds
Cgl0004I processed model has 313 rows, 616 columns (616 integer (616 of which binary)) and 1626 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.984615
Cb